#### Imports 

In [1]:
from sagemaker import get_execution_role
from time import gmtime, strftime
import pandas as pd
import sagemaker
import boto3
import time

#### Essentials

In [2]:
bucket = sagemaker.Session().default_bucket()
prefix = 'loan-default-prediction'
region = 'us-east-1'

In [3]:
batch_input = f's3://{bucket}/{prefix}/batch_input/'
batch_input

's3://sagemaker-us-east-1-892313895307/loan-default-prediction/batch_input/'

In [4]:
batch_output = f's3://{bucket}/{prefix}/batch_output/'
batch_output

's3://sagemaker-us-east-1-892313895307/loan-default-prediction/batch_output/'

In [5]:
current_timestamp = strftime('%Y-%m-%d-%H-%M-%S', gmtime())

In [6]:
training_job_name = 'tuning-job-1-fe12f12c5a7642a59c-186-56633a3d'  # Copy this from the console
model_name = f'autopilot-best-model-{current_timestamp}'
batch_job_name = f'autopilot-batch-job-{current_timestamp}'

In [7]:
session = boto3.Session()
sagemaker_execution_role = get_execution_role()
sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client('sagemaker', region_name=region)
s3_client = boto3.client('s3')

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210527T121473 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


#### Copy batch input data from local to S3

In [8]:
!aws s3 cp ./data/train/loans_unlabeled.csv {batch_input}

upload: data/train/loans_unlabeled.csv to s3://sagemaker-us-east-1-892313895307/loan-default-prediction/batch_input/loans_unlabeled.csv


#### Create a Model object using previously run training job name

In [9]:
info = sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
info

{'TrainingJobName': 'tuning-job-1-fe12f12c5a7642a59c-186-56633a3d',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:892313895307:training-job/tuning-job-1-fe12f12c5a7642a59c-186-56633a3d',
 'TuningJobArn': 'arn:aws:sagemaker:us-east-1:892313895307:hyper-parameter-tuning-job/tuning-job-1-fe12f12c5a7642a59c',
 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:892313895307:automl-job/loan-default-prediction-3',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-autopilot-experiments/outputs/loan-default-prediction-3/tuning/loan-defau-dpp3-xgb/tuning-job-1-fe12f12c5a7642a59c-186-56633a3d/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'_kfold': '5',
  '_tuning_objective_metric': 'validation:f1',
  'alpha': '0.01487797714947976',
  'colsample_bytree': '0.7366941420147759',
  'eta': '0.0062042369988801185',
  'gamma': '0.04353380090583981',
  'lambda': '0.006606090044497277',
  'max_depth': '7',
  'min_child_weight': '3.02916850

In [10]:
model_artifact_url = info['ModelArtifacts']['S3ModelArtifacts']
model_artifact_url

's3://sagemaker-autopilot-experiments/outputs/loan-default-prediction-3/tuning/loan-defau-dpp3-xgb/tuning-job-1-fe12f12c5a7642a59c-186-56633a3d/output/model.tar.gz'

In [11]:
container_uri = info['AlgorithmSpecification']['TrainingImage']

In [12]:
primary_container = {
    'Image': container_uri,
    'ModelDataUrl': model_artifact_url
  }

In [13]:
response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=sagemaker_execution_role,
    PrimaryContainer=primary_container)

In [14]:
response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:892313895307:model/autopilot-best-model-2021-06-28-19-05-17',
 'ResponseMetadata': {'RequestId': 'f380b18b-f8fd-4c34-9228-68fe84b93fb1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f380b18b-f8fd-4c34-9228-68fe84b93fb1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '102',
   'date': 'Mon, 28 Jun 2021 19:05:18 GMT'},
  'RetryAttempts': 0}}

#### Create a Batch Transformer for Inference

In [15]:
request = {
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "BatchStrategy": "MultiRecord",
    "TransformOutput": {
        "S3OutputPath": batch_output
    },
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": batch_input 
            }
        },
        "ContentType": "text/csv",
        "SplitType": "Line",
        "CompressionType": "None"
    },
    "TransformResources": {
            "InstanceType": "ml.m5.xlarge",
            "InstanceCount": 1
    }
}

In [16]:
response = sagemaker_client.create_transform_job(**request)
response

{'TransformJobArn': 'arn:aws:sagemaker:us-east-1:892313895307:transform-job/autopilot-batch-job-2021-06-28-19-05-17',
 'ResponseMetadata': {'RequestId': 'fadde3f7-aa27-447a-8b23-2694760824db',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fadde3f7-aa27-447a-8b23-2694760824db',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '116',
   'date': 'Mon, 28 Jun 2021 19:05:19 GMT'},
  'RetryAttempts': 0}}

In [17]:
while True:
    response = sagemaker_client.describe_transform_job(TransformJobName=batch_job_name)
    status = response['TransformJobStatus']
    if  status == 'Completed':
        print("Transform job ended with status: {}".format(status))
        break
    if status == 'Failed':
        message = response['FailureReason']
        print('Transform failed with the following error: {}'.format(message))
        raise Exception('Transform job failed') 
    print("Transform job is still in status: {}".format(status))    
    time.sleep(30) 

Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform failed with the following error: ClientError: See job logs for more information


Exception: Transform job failed